In [3]:
%matplotlib inline
from vnpy.trader.vtObject import VtBarData
from vnpy.trader.vtConstant import EMPTY_STRING
from vnpy.trader.app.ctaStrategy.ctaTemplate import CtaTemplate, BarManager
from vnpy.trader.app.ctaStrategy.ctaBase import TICK_DB_NAME
from datetime import time
from vnpy.trader.app.ctaStrategy.ctaBacktesting import BacktestingEngine

In [6]:
#合约代码
code = '600380'
exchange = 'SSE'
vtCode = '.'.join([code, exchange])

In [38]:
'''
课程策略
'''
class DualThrustStrategy(CtaTemplate):
    """DualThrust交易策略"""
    className = 'DualThrustStrategy'
    author = u'用Python的交易员'

    # 策略参数
    fixedSize = 100
    k1 = 0.4
    k2 = 0.6

    initDays = 10

    # 策略变量
    barList = []                # K线对象的列表

    dayOpen = 0
    dayHigh = 0
    dayLow = 0
    
    range = 0
    longEntry = 0
    shortEntry = 0
    exitTime = time(hour=14, minute=55)

    longEntered = False
    shortEntered = False

    # 参数列表，保存了参数的名称
    paramList = ['name',
                 'className',
                 'author',
                 'vtSymbol',
                 'k1',
                 'k2']    

    # 变量列表，保存了变量的名称
    varList = ['inited',
               'trading',
               'pos',
               'range',
               'longEntry',
               'shortEntry',
               'exitTime']  

    #----------------------------------------------------------------------
    def __init__(self, ctaEngine, setting):
        """Constructor"""
        super(DualThrustStrategy, self).__init__(ctaEngine, setting) 
        
        self.bm = BarManager(self.onBar)
        self.barList = []
        
        self.a = True

    #----------------------------------------------------------------------
    def onInit(self):
        """初始化策略（必须由用户继承实现）"""
        self.writeCtaLog(u'%s策略初始化' %self.name)
    
        # 载入历史数据，并采用回放计算的方式初始化策略数值
        initData = self.loadBar(self.initDays)
        for tick in initData:
            self.onTick(tick)

        self.putEvent()

    #----------------------------------------------------------------------
    def onStart(self):
        """启动策略（必须由用户继承实现）"""
        self.writeCtaLog(u'%s策略启动' %self.name)
        self.putEvent()

    #----------------------------------------------------------------------
    def onStop(self):
        """停止策略（必须由用户继承实现）"""
        self.writeCtaLog(u'%s策略停止' %self.name)
        self.putEvent()

    #----------------------------------------------------------------------
    def onTick(self, tick):
        """收到行情TICK推送（必须由用户继承实现）"""
        if tick.datetime.time() > self.exitTime:
            print '======%s======' % tick.datetime
            print tick.lastPrice
        self.bm.updateTick(tick)
        
    #----------------------------------------------------------------------
    def onBar(self, bar):
        """收到Bar推送（必须由用户继承实现）"""
        # 撤销之前发出的尚未成交的委托（包括限价单和停止单）
        self.cancelAll()
        
        # 计算指标数值
        self.barList.append(bar)
        
        if len(self.barList) < 2:
            self.dayOpen = bar.open
            self.dayHigh = bar.high
            self.dayLow = bar.low
            return
        
        if len(self.barList) > 2:
            self.barList.pop(0)
            
        lastBar = self.barList[-2]
        # 新的一天
        if lastBar.datetime.date() != bar.datetime.date():
            #print'======%s======' %  bar.datetime
            #print 'pos = %ld' % self.pos
            #print '\n'
            # 如果已经初始化
            if self.dayHigh:
                self.range = self.dayHigh - self.dayLow
                self.longEntry = bar.open + self.k1 * self.range
                self.shortEntry = bar.open - self.k2 * self.range           
            
            self.dayOpen = bar.open
            self.dayHigh = bar.high
            self.dayLow = bar.low

            self.longEntered = False
            self.shortEntered = False
        else:
            self.dayHigh = max(self.dayHigh, bar.high)
            self.dayLow = min(self.dayLow, bar.low)

        #第一天或者前一天没有波动
        if not self.range:
            return
        
        #尚未到收盘
        if bar.datetime.time() < self.exitTime:
            if self.pos == 0:
                if bar.close > self.dayOpen:
                    if not self.longEntered:
                        self.buy(self.longEntry, self.fixedSize, stop=True)
                else:
                    if not self.shortEntered:
                        self.short(self.shortEntry, self.fixedSize, stop=True)
    
            # 持有多头仓位
            elif self.pos > 0:
                self.longEntered = True

                # 多头止损单
                self.sell(self.shortEntry, self.fixedSize, stop=True)
                
                # 空头开仓单
                if not self.shortEntered:
                    self.short(self.shortEntry, self.fixedSize, stop=True)
                
            # 持有空头仓位
            elif self.pos < 0:
                self.shortEntered = True

                # 空头止损单
                self.cover(self.longEntry, self.fixedSize, stop=True)
                
                # 多头开仓单
                if not self.longEntered:
                    self.buy(self.longEntry, self.fixedSize, stop=True)
            
        # 收盘平仓
        else:
            print'======%s======' %  bar.datetime
            if self.pos > 0:
                print 'sell (price = %.2f  pos = %ld)' % (bar.close*0.99, abs(self.pos))
                self.sell(bar.close * 0.99, abs(self.pos), stop = True)
            elif self.pos < 0:
                print 'cover (price = %.2f  pos = %ld)' % (bar.close*1.01, abs(self.pos))
                self.cover(bar.close * 1.01, abs(self.pos), stop = True)
 
        # 发出状态更新事件
        self.putEvent()

    #----------------------------------------------------------------------
    def onOrder(self, order):
        """收到委托变化推送（必须由用户继承实现）"""
        pass

    #----------------------------------------------------------------------
    def onTrade(self, trade):
        # 发出状态更新事件
        theBar = self.barList[-1]
        '''
        print theBar.datetime.date()
        print '===========%s=========='  % trade.dt
        print 'direction:%s' % trade.direction
        print 'offset:%s' % trade.offset
        print 'price:%s' % trade.price
        print '\n'
        '''
        
        self.putEvent()
        
    #----------------------------------------------------------------------
    def onStopOrder(self, so):
        """停止单推送"""
        pass

In [39]:
'''
回测
'''
#初始化数据需要的自然日天数
initDays = 3
#初始资金
DualThrustStrategy.capital = 1000000

#创建回测引擎实例
engine = BacktestingEngine()

#设置引擎的回测模式为K线
engine.setBacktestingMode(engine.TICK_MODE)

#设置回测的起始日期
engine.setStartDate('20170101', initDays = initDays) #这里的initDays是真正起作用的，20天是指自然日肯定不够的

#设置产品相关参数
engine.setSlippage(0)           #滑点设置为0
engine.setRate(1.5/10000)       #股票交易手续费万1.5
engine.setSize(1)               #股票每股为1
engine.setPriceTick(0.01)      #股票最小价格变动
engine.setCapital(DualThrustStrategy.capital) #初始资金

#设置使用的数据库
engine.setDatabase(TICK_DB_NAME, vtCode)

#引擎中创建策略对象
engine.initStrategy(DualThrustStrategy, {})

#开始跑回测
engine.runBacktesting()

#显示回测结果
#engine.showDailyResult()

2017-11-12 21:57:49.277000	开始载入数据
2017-11-12 21:57:50.119000	载入完成，数据量：308779
2017-11-12 21:57:50.119000	开始回测
======2017-01-03 14:55:05======
10.01
======2017-01-03 14:55:08======
10.0
======2017-01-03 14:55:17======
10.0
======2017-01-03 14:55:23======
10.0
======2017-01-03 14:55:26======
10.0
======2017-01-03 14:55:32======
10.01
======2017-01-03 14:55:47======
10.02
======2017-01-03 14:55:50======
10.01
======2017-01-03 14:55:59======
10.02
======2017-01-03 14:56:02======
10.01
======2017-01-03 14:56:11======
10.01
======2017-01-03 14:56:14======
10.01
======2017-01-03 14:56:20======
10.01
======2017-01-03 14:56:26======
10.01
======2017-01-03 14:56:35======
10.01
======2017-01-03 14:56:41======
10.02
======2017-01-03 14:56:47======
10.01
======2017-01-03 14:56:50======
10.01
======2017-01-03 14:56:56======
10.01
======2017-01-03 14:57:02======
10.02
======2017-01-03 14:57:11======
10.02
======2017-01-03 14:57:14======
10.02
======2017-01-03 14:57:26======
10.01
======2017-01-03 14:5

In [26]:
dic = {'a':1, 'b':2}
print dic
dic.clear()
print dic

{'a': 1, 'b': 2}
{}


In [8]:
a = 0.123456
b = round(a, 2)
print b

0.12
